<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [4]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.0001
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
neurons = 2

#--------------------------------------------------------------------

#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#--------------------------------------------------------------------

def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

In [27]:
def deltaRule(data, level = 1, N = 2, eps = 0.01):
  Kmax = 1                                #максимальное количество итераций
  w0 = [[randomNumber()] * N] * N         #синаптические веса

  #wi = [[0] * N] * N
  si = [0] * N
  yi = [0] * N
  level_i = 0
  k = 0
  while(level_i < level):
    while(k < Kmax):
      wi = w0
      #добавить цикл по строкам таблицы data
      for i in range(N):
        for j in range(N):
          #print(data[data.columns[j]][i])
          #data нужно иначе задать
          si[i] += data[data.columns[j]][i] * wi[j][i]


        #добавить расчёт эпсилон

        #print('s' + str(i+1) + ' = ' + str(si[i]))
        yi[i] = hyperbolicTangentFunction(si[i])
        #print('y' + str(i+1) + ' = ' +  str(yi[i]))


      #пересчёт w


      k += 1
      #вместо контроля количества итераций, лучше сравнивать по СКО


    level_i += 1


  return 0

In [28]:
functions = [hyperbolicTangentFunction, thresholdFunction]

table = pd.DataFrame(it.product([0,1], repeat=2), columns=['X1', 'X2'])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(table, layer, neurons, learningRate)

s1 = 0.0
y1 = 0.0
s2 = 0.12383326830238606
y2 = 0.06183763265619088


0

In [24]:
table

,X1,X2,Y1 = equivalence (~),Y2 = conjunction (^)
0,0,0,1,0
1,0,1,0,0
2,1,0,0,0
3,1,1,1,1
